In [1]:
!pip install pytorch-tabnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 1.4 MB/s eta 0:00:00


In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
import os
import requests


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

from pytorch_tabnet.pretraining import TabNetPretrainer
from pytorch_tabnet.tab_model import TabNetClassifier
import torch

In [4]:
train_data_main = pd.read_parquet('/kaggle/input/datasets/jorapro/data-fusion-contest-2026-task-2-data/train_extra_features.parquet')
test_data_main = pd.read_parquet('/kaggle/input/datasets/jorapro/data-fusion-contest-2026-task-2-data/test_extra_features.parquet')

In [5]:
train_data_main.head()

,customer_id,num_feature_133,num_feature_134,num_feature_135,num_feature_136,num_feature_137,num_feature_138,num_feature_139,num_feature_140,num_feature_141,...,num_feature_2364,num_feature_2365,num_feature_2366,num_feature_2367,num_feature_2368,num_feature_2369,num_feature_2370,num_feature_2371,num_feature_2372,num_feature_2373
0,1000001,-0.116197,-0.009981,NaN,NaN,NaN,NaN,-0.067005,NaN,NaN,...,NaN,-0.002760,NaN,0.0,-0.030607,-0.015407,-0.404816,-0.395589,NaN,0.0
1,1000002,-0.116197,-0.009981,NaN,NaN,NaN,NaN,-0.067005,-0.088452,NaN,...,NaN,-0.001345,NaN,0.0,-0.030607,-0.047964,1.630249,-0.395589,-0.201813,0.0
2,1000003,-0.116197,-0.009981,NaN,NaN,NaN,NaN,-0.067005,-0.088452,NaN,...,NaN,-0.002760,NaN,0.0,-0.030607,-0.045637,-0.404816,-0.395589,-0.271623,0.0
3,1000004,-0.116197,-0.009981,NaN,NaN,NaN,NaN,-0.067005,NaN,NaN,...,NaN,-0.002760,NaN,0.0,-0.030607,-0.058314,-0.404816,NaN,NaN,0.0
4,1000005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.088452,NaN,...,NaN,-0.002760,NaN,0.0,-0.030607,-0.043266,-0.404816,0.048602,0.206248,0.0


In [ ]:
too_many_nan = (train_data_main.isna()/train_data_main.shape[0])>0.5

In [4]:
train_data_main = pd.read_parquet('/kaggle/input/datasets/jorapro/data-fusion-contest-2026-task-2-data/train_main_features.parquet')
train_target = pd.read_parquet('/kaggle/input/datasets/jorapro/data-fusion-contest-2026-task-2-data/train_target.parquet')
test_data_main = pd.read_parquet('/kaggle/input/datasets/jorapro/data-fusion-contest-2026-task-2-data/test_main_features.parquet')

train_data = pd.merge(train_data_main, train_target, on='customer_id', how='inner')

In [5]:
train_columns = list(train_data_main.columns)
cat_features = train_columns[1:68]
num_features = train_columns[68:]
train_features = train_columns[1:]
target_features = train_target.columns[1:]

X = train_data[train_features]#.dropna(axis=1)
Y = train_data[target_features]

X_test = test_data_main[X.columns]

In [6]:
import pandas as pd
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

# 1. Простая стратегия (Медиана)
imputer_simple = SimpleImputer(strategy='median')

imputer_simple.fit(X[num_features])

X.loc[:,num_features] = imputer_simple.transform(X[num_features])
X_test.loc[:,num_features] = imputer_simple.transform(X_test[num_features])


/tmp/ipykernel_167/2228836090.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.loc[:,num_features] = imputer_simple.transform(X[num_features])
/tmp/ipykernel_167/2228836090.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.loc[:,num_features] = imputer_simple.transform(X_test[num_features])


In [7]:
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, shuffle=True)

In [8]:
X_train_scaled = X_train # Обучение и трансформация
X_val_scaled = X_val # Обучение и трансформация
X_test_scaled = X_test # Обучение и трансформация

In [9]:
scaler = StandardScaler()
X_train_scaled[num_features] = scaler.fit_transform(X_train[num_features]) # Обучение и трансформация
X_val_scaled[num_features] = scaler.fit_transform(X_val[num_features]) # Обучение и трансформация
X_test_scaled[num_features] = scaler.transform(X_test[num_features])       # Трансформация теста

/tmp/ipykernel_167/3100060260.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_scaled[num_features] = scaler.transform(X_test[num_features])       # Трансформация теста


In [10]:
unique_cols = (X[cat_features].max(axis=0).values+1).astype(int).tolist()

In [11]:
# # 1. Инициализация претрейнера
# unsupervised_model = TabNetPretrainer(
#     n_d = 12,
#     n_a = 12,
#     n_steps = 4,
#     gamma = 1.3,
#     optimizer_fn=torch.optim.AdamW,
#     optimizer_params=dict(lr=2e-2),
#     cat_idxs=[i for i in range(67)],
#     cat_dims=unique_cols,
#     cat_emb_dim=4,
#     verbose=2,
#     mask_type='sparsemax' # или 'entmax' — определяет, насколько "жестко" выбирать признаки
# )

In [12]:
# # 2. Обучение (восстановление признаков)
# # Здесь X_train — это ваши категориальные данные, переведенные в Ordinal (числа 0, 1, 2...)
# unsupervised_model.fit(
#     X_train=X_train.values,
#     eval_set=[X_val.values],
#     pretraining_ratio=0.8, # какой % данных маскировать для восстановления
#     weights=0,
#     max_epochs=50,
#     patience=10,
#     batch_size=1024,
#     virtual_batch_size=128,
#     num_workers=0,
#     drop_last=True,
#     callbacks=None,
#     pin_memory=True,
#     warm_start=False,
# )

In [13]:
# # 3. Извлечение сжатых признаков (эмбеддингов)
# # Это и есть ваш "энкодер"
# embedded_X, _ = unsupervised_model.predict(X_train)

In [14]:
from pytorch_tabnet.multitask import TabNetMultiTaskClassifier # <--- Важное изменение

In [15]:
clf = TabNetMultiTaskClassifier(
    cat_idxs=[i for i in range(67)],
    cat_dims=unique_cols,
    cat_emb_dim=4,
    n_d=64, n_a=64, # Размерность эмбеддинга будет n_d (64)
    n_steps = 2,
    gamma = 1.3,
    n_independent = 2,
    n_shared = 2,
    optimizer_fn=torch.optim.AdamW,
    optimizer_params=dict(lr=2e-2),
    scheduler_params={"T_max":50, "eta_min":0.001},
    scheduler_fn=torch.optim.lr_scheduler.CosineAnnealingLR,
    mask_type='sparsemax',
)

/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


In [16]:
# Обучаем сразу на 42 таргета
# Y_train должен быть (n_samples, 42) из 0 и 1
clf.fit(
    X_train=X_train_scaled.values, y_train=y_train.values,
    eval_set=[(X_val_scaled.values, y_val.values)],
    eval_metric=['auc'], # или 'auc'
    max_epochs=50,
    batch_size=16384, virtual_batch_size=4096
)

epoch 0  | loss: 0.1076  | val_0_auc: 0.68876 |  0:00:21s
epoch 1  | loss: 0.09404 | val_0_auc: 0.72225 |  0:00:42s
epoch 2  | loss: 0.09279 | val_0_auc: 0.73792 |  0:01:04s
epoch 3  | loss: 0.09223 | val_0_auc: 0.74217 |  0:01:25s
epoch 4  | loss: 0.09177 | val_0_auc: 0.7446  |  0:01:46s
epoch 5  | loss: 0.09147 | val_0_auc: 0.74828 |  0:02:08s
epoch 6  | loss: 0.09115 | val_0_auc: 0.75014 |  0:02:29s
epoch 7  | loss: 0.09089 | val_0_auc: 0.75711 |  0:02:50s
epoch 8  | loss: 0.09063 | val_0_auc: 0.75734 |  0:03:12s
epoch 9  | loss: 0.09047 | val_0_auc: 0.76021 |  0:03:33s
epoch 10 | loss: 0.09024 | val_0_auc: 0.76187 |  0:03:54s
epoch 11 | loss: 0.0901  | val_0_auc: 0.76084 |  0:04:15s
epoch 12 | loss: 0.08995 | val_0_auc: 0.76386 |  0:04:37s
epoch 13 | loss: 0.08971 | val_0_auc: 0.76482 |  0:04:58s
epoch 14 | loss: 0.08956 | val_0_auc: 0.76761 |  0:05:19s
epoch 15 | loss: 0.08942 | val_0_auc: 0.77064 |  0:05:40s
epoch 16 | loss: 0.08917 | val_0_auc: 0.77659 |  0:06:01s
epoch 17 | los

/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [18]:
# Сохраняем в файл (создастся zip-архив)
save_history = clf.save_model('my_tabnet_multitask_model')

Successfully saved model at my_tabnet_multitask_model.zip


In [19]:
save_history

'my_tabnet_multitask_model.zip'

In [43]:
# ИЗВЛЕЧЕНИЕ ЭМБЕДДИНГОВ:
# Метод predict в TabNetClassifier возвращает предсказания классов,
# но мы можем достать латентный слой через метод .predict_soft_embeddings()
embeddings_train = clf.predict_proba(X_val_scaled.values)
val_preds = np.array(embeddings_train)[:,:,1]

In [44]:
roc_auc = roc_auc_score(y_val, val_preds.T, average='macro')
roc_auc

np.float64(0.7894584328127371)

In [45]:
test_preds = clf.predict_proba(X_test_scaled.values)

In [46]:
ts_pr = np.array(test_preds)[:,:,1]

In [47]:
ts_pr.shape

(41, 250000)

In [49]:
sample_sub = pd.read_parquet('/kaggle/input/datasets/jorapro/data-fusion-contest-2026-task-2-data/sample_submit.parquet')

In [50]:
sample_sub.iloc[:,1:] = ts_pr.T

In [51]:
sample_sub.to_parquet('sub_TabNet.parquet', index=False)